In [1]:
import os
import pandas as pd

### Import listings files

In [2]:
directory = '/Users/Asus/Desktop/Unity Game/python/Data_Analysis/WebScrapping/Abnb/Scraping_lists'

df = pd.DataFrame()
for file in os.listdir(directory):
    df = df.append((pd.read_csv(os.path.join(directory, file))), ignore_index=True)

### FIrst glance of data

In [3]:
df.head()

,Room type,Name,Descriptions,Rating,Price
0,Entire serviced apartment in Kuala Lumpur,LUXURY Studio with NETFLIX & SKY POOL at CEYLONZ,"4 guests, Studio, 1 bed, 1 bath, Wifi, Air con...",4.86,$19 per night
1,Entire serviced apartment in Kuala Lumpur,★Studio★KL★JLN TUN RAZAK★10min KLCC★IJN★TV Box★,"2 guests, Studio, 1 bed, 1 bath, Wifi, Air con...",4.77,$26 per night
2,Entire rental unit in Kuala Lumpur,Staycation for Fully Vaccinated Guest. KLCC area,"2 guests, 1 bedroom, 1 bed, 1 bath, Wifi, Air ...",4.95,$35 per night
3,Entire serviced apartment in Kuala Lumpur,"350M TO KLCC,VORTEX HIGH FLOOR KLCC VIEW@BLACK...","6 guests, 3 bedrooms, 3 beds, 2 baths, Wifi, A...",NaN,$11 per night
4,Hotel room in Kuala Lumpur,Superior Queen Room,"2 guests, 1 bedroom, 1 bed, 1 private bath, Wi...",4.51,$13 per night


### Checking null value in all columns, and drop the row

In [4]:
# checking null row
df[df.isnull().any(axis=1)]
# drop null price row / drop all null values in row
df.dropna(how='all', inplace=True)

### Remove duplicate row

In [5]:
# remove duplicate row
duplicate = df[df['Name'].duplicated(keep=False)]
df.drop_duplicates(keep='first', inplace=True)
df = df.reset_index()

### Checking row number and Null/ empty row

In [6]:
print(f'Length of row {len(df)}, number of null:\n{df.isnull().sum()}')

Length of row 202, number of null:
index           0
Room type       0
Name            0
Descriptions    0
Rating          6
Price           0
dtype: int64


### New features -  Location
- to identify listing area

In [7]:
df['Room type'] = df['Room type'].str.replace(' in ', ',')
df['Location'] = df['Room type'].apply(lambda x: str(x).split(',')[-1])
df['Location'] = df['Location'].str.strip()
df.Location.value_counts()

Kuala Lumpur                         160
Federal Territory of Kuala Lumpur      9
Cheras                                 7
Petaling Jaya                          6
Bukit Bintang                          5
Balakong                               3
Kuala Kubu Baru                        3
Seri Kembangan                         2
Batu 9 Cheras                          2
Kampung Baru                           1
Hulu Langat                            1
i-City                                 1
Sri Hartamas                           1
Cyberjaya                              1
Name: Location, dtype: int64

### New features -  Room
- to remove location, keep only room type info

In [8]:
df['Room'] = df['Room type'].apply(lambda x : x.split(',')[0])
df.Room.value_counts()

Entire rental unit            92
Entire condominium (condo)    59
Entire serviced apartment     39
Entire villa                   4
Entire loft                    4
Private room                   2
Hotel room                     2
Name: Room, dtype: int64

### New features - Guests
- Number of guest

In [9]:
df['Guests'] = [guest.split(',')[0].split(' ')[0] if 'guests' in guest.split(',')[0] else None for guest in df.Descriptions]
df.Guests.value_counts()

4     76
2     72
3     38
5      6
6      6
7      2
16     1
10     1
Name: Guests, dtype: int64

### New features - Bed type

In [10]:
df['bed_type'] = [type.split(',')[1] for type in df.Descriptions]
df.bed_type.value_counts()

 1 bedroom     98
 Studio        96
 2 bedrooms     4
 3 bedrooms     3
 8 bedrooms     1
Name: bed_type, dtype: int64

### New features - Number of bed

In [11]:
df['Number_of_bed'] = [bed.split(',')[2].split(' ')[1] if 'bed' in bed.split(',')[2] else None for bed in df.Descriptions]
df.Number_of_bed.value_counts()

1     149
2      35
3       5
4       4
0       3
6       2
12      1
8       1
Name: Number_of_bed, dtype: int64

### New features - Number of bath

In [12]:
df['Number_of_bath'] = [bath.split(',')[3] if 'bath' in bath.split(',')[3] else None for bath in df.Descriptions]
df.Number_of_bath.value_counts()

 1 bath            159
 1.5 baths          24
 2 baths             9
 1 private bath      4
 Half-bath           2
 7 baths             1
 3 baths             1
Name: Number_of_bath, dtype: int64

### New features - Wifi, Air Cond, Kitchen, Self Check-in

In [13]:
df['wifi'] = [True if 'Wifi' in wifi else False for wifi in df.Descriptions]
df['Air_cond'] = [True if 'Air conditioning' in ac else False for ac in df.Descriptions]
df['Kitchen'] = [True if 'Kitchen' in Kitchen else False for Kitchen in df.Descriptions]
df['Self_check_in'] = [True if 'Self check-in' in sci else False for sci in df.Descriptions]

In [14]:
us_to_myr = 4.5

df['Price_in_MYR'] = df.Price.apply(lambda x: int(x.split(' ')[0].replace('$', '')) * us_to_myr)
# to double confirm if every listing is per night
unit_price = df.Price.apply(lambda x: ' '.join(x.split(' ')[1:]))
unit_price.value_counts()

per night    202
Name: Price, dtype: int64

In [17]:
useful_features = ['Location', 'Room', 'Guests', 'bed_type', 'Number_of_bed', 'Number_of_bath', 'wifi', 'Air_cond', 'Kitchen', 'Self_check_in', 'Price_in_MYR']
data = df[useful_features]
data.head()

,Location,Room,Guests,bed_type,Number_of_bed,Number_of_bath,wifi,Air_cond,Kitchen,Self_check_in,Price_in_MYR
0,Kuala Lumpur,Entire serviced apartment,4,Studio,1,1 bath,True,True,True,True,85.5
1,Kuala Lumpur,Entire serviced apartment,2,Studio,1,1 bath,True,True,True,False,117.0
2,Kuala Lumpur,Entire rental unit,2,1 bedroom,1,1 bath,True,True,True,True,157.5
3,Kuala Lumpur,Entire serviced apartment,6,3 bedrooms,3,2 baths,True,True,True,False,49.5
4,Kuala Lumpur,Hotel room,2,1 bedroom,1,1 private bath,True,True,False,True,58.5


In [16]:
# df.sort_values(by='Number_of_bath')